In [162]:
import cv2 
import numpy as np

# 算法概要

##### 根据所用算法的需求 使用膨胀运算 通过[[0,1,0],[0,1,0],[0,1,0]]的内核 

##### 使得要取出的字母 上下连接成一个整体 同时左右有相对独立

##### 分析易知图片边缘存在一定的无效信息

##### 取得图像中轮廓 筛出 存在轮廓坐标超出宽20%-80% 高10%-90%范围内的轮廓 

##### 为避免 筛除后仍然存在极端值(大面积的外围轮廓 可忽略不计的小轮廓) 再进行一次极值去除

### 针对处理过后的轮廓 根据字母数字的高和宽具有一定的一致性 做一次聚类处理

### 采用 K均值的算法 按照长和宽形成的二维数据 进行一次聚类操作

### 同时再根据空间坐标关系 以及轮廓关系来确保归类的准确度

### 从而实现不采用模板 自动实现识别字母的结果

## 

#### 以下为聚类处理之后 针对特殊情况的处理

#### 由于要识别的内容中 存在冒号 而冒号的长和宽与字母数字有较大差异 很有可能会被分为另一类

####  同时可能左上角 右下角存在高宽与字母相近的轮廓

#### 通过 轮廓关系和坐标位置关系来 修改这些值的tag 来避免误判(后面有详细规则定义) 

具体的实现方式可参照下面的代码

### 读入图片数据 

In [163]:
image = cv2.imread("image/c1/chinatel.jpg")

In [164]:
img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

In [165]:
ret,binary = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)

## 基于之后的算法考虑 需要保证整个字母为一个连续轮廓 所以对进行预处理 详细原因见之后分类运算

## 使用膨胀运算 以及 [[0,1,0],[0,1,0],[0,1,0]]的内核 使整个字母上下连接 以保证为连续轮廓 同时左右相对独立

In [166]:
kernel = np.array([[0,1,0],[0,1,0],[0,1,0]],dtype=np.uint8)

In [167]:
dialte_binary = cv2.dilate(binary,kernel,iterations = 2)

# 筛选出所需要的轮廓

### 截取 所有的轮廓坐标 都位于原图像 宽20%-80% 高10%-90%范围内的轮廓

### 具体实现：采用numpy 布尔索引 当该轮廓各坐标点全部位于范围内时 向mod_con中添加该轮廓 mod_hie中更新对应关系

In [168]:
contours,hierachy =  cv2.findContours(dialte_binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [169]:
copy = image.copy()
r = cv2.drawContours(copy,contours,-1,(0,0,255),2)

In [170]:
height,width = image.shape[:2]

In [171]:
mod_con = []  # 初始化筛选后的轮廓列表
mod_hie = []  # 对应轮廓关系列表
for i,con in enumerate(contours):
    new_con = con.reshape(-1, 2)  # 转换为二维
    # 截取 所有的轮廓坐标 都位于原图像 宽20%-80% 高10%-90%范围内的轮廓
    # 采用numpy 布尔索引 当该轮廓各坐标点全部位于范围内时 向mod_con中添加该轮廓 mod_hie中更新对应关系
    flag = np.all(new_con[:, 1] > height * 0.1) and np.all(new_con[:1] < height * 0.9) and np.all(
        new_con[:, 0] > width * 0.2) and np.all(new_con[:, 0] < width * 0.8)
    if flag:
        mod_con.append(con)
        mod_hie.append([hierachy[0][i]])        
len(mod_hie)
len(mod_con)

102

In [172]:
copy2 = image.copy()
img_gray2 = cv2.cvtColor(copy2,cv2.COLOR_BGR2GRAY)
ret2,binary2 = cv2.threshold(img_gray2,0,255,cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
r2 = cv2.drawContours(copy2,mod_con,-1,(0,0,255),2)

### 针对这幅图的话 在筛选过后 其实已经不存在极端值 不过考虑到代码的健壮性 
### 假设在第一次筛选之后 仍然存在极端异常值（大面积的外围轮廓 可忽略不计的小轮廓）

## 去除极端值

In [173]:
areaslist = [cv2.contourArea(mod_con[i]) for i,contour in enumerate(mod_con)]

### 去除异常极大值 如果最大值大于所有值之和的一半 则去除

In [174]:
for i in range(len(areaslist)):
    max_area = max(areaslist)
    if max_area > sum(areaslist)/2:
        removeid = areaslist.index(max_area)
        del areaslist[removeid]
        del mod_hie[removeid]
        mod_con = np.delete(mod_con,removeid)

### 去除极小值 在去除极大值之后 面积最小值小于均值的十分之一的值 去除

In [175]:
for i in range(len(areaslist)):
    min_area = min(areaslist)
    if min_area < sum(areaslist,0.0)/len(areaslist)*0.1:
        removeid = areaslist.index(min_area)
        del areaslist[removeid]
        del mod_hie[removeid]
        mod_con = np.delete(mod_con,removeid)

# 根据字母数字的高和宽具有一定的一致性 做一次聚类处理

# 使用K近邻算法来实现对于不同的轮廓的聚类

# 这也是在预处理时选择上下膨胀的原因 使得字母为连续轮廓 同时左右相对独立

## 取出轮廓的矩形轮廓

#### 去除子轮廓

In [176]:
for i,hie in enumerate(mod_hie):
    if hie[0][3]!=-1:
        del mod_hie[i]
        mod_con = np.delete(mod_con,i)

In [177]:
rects =[]
wh=[]
areas =[]
for i,contour in enumerate(mod_con):
    x,y,w,h = cv2.boundingRect(mod_con[i])
    area = cv2.contourArea(mod_con[i])
    wh.append([w,h])
    areas.append(area)
    rects.append([x,y,w,h])

In [178]:
rects = np.array(rects)

In [179]:
areas = np.array(areas)

In [180]:
mod_hie =np.array(mod_hie)

In [181]:
from sklearn.cluster import KMeans

In [182]:
clf = KMeans(n_clusters=2,init = "random")

In [183]:
clf.fit(wh)

KMeans(algorithm='auto', copy_x=True, init='random', max_iter=300, n_clusters=2,
       n_init=10, n_jobs=None, precompute_distances='auto', random_state=None,
       tol=0.0001, verbose=0)

In [184]:
label = clf.labels_

### 分类后 取得面积更大的一类

In [185]:
if np.sum(areas[label==0]) > np.sum(areas[label==1]):
    tag = 0
else :
    tag = 1

## 由于要识别的内容中 存在冒号 而冒号的长和宽与字母数字有较大差异

## 很有可能会被分为另一类 同时可能左上角 右下角存在高宽与字母相近的轮廓

## 通过 轮廓关系和坐标位置关系来 修改这些值的label 来避免误判

### 若前(后)轮廓都为 tag类 且其顶点或底点坐标位置 与前后顶点或底点坐标 基本位于同一水平线上 则更改其标记为 tag 

### 根据空间坐标关系更改label (同一行最远坐标不超过下列坐标width的三分之二 最小坐标值超过上列坐标的三分之一)

其实应该用均值或方差来自动限制行差 但是不想写了 直接给个定值 有机会再维护吧 

In [186]:
row = [0]
for i in range(1,len(rects[label == tag])-1):
    if rects[label == tag][i-1][1] - rects[label == tag][i][1] > 15:
        row.append(i)
row.append(len(rects[label==tag])-1)

In [187]:
label_ture = []
tag_re = 0  # 增加一个计数单位
tag_ver = 1 - tag  # 取得与tag相反的分类
for i in range(len(label)-1):
    if label[i] == tag_ver:
        if i == 0:
            label_ture.append(label[i])
            continue
    if i == len(label):
        label_ture.append(label[i])
        continue
    if label[i - 1] == tag and label[i + 1] == tag and (
            ((rects[i - 1][2] - rects[i][2] < 20) and (rects[i][2] - rects[i + 1][2] < 20)) or (
            (abs((rects[i - 1][2] + rects[i - 1][3]) - (rects[i][2] + rects[i - 1][3])) < 10) and (
            abs((rects[i][2] + rects[i - 1][3]) - (rects[i + 1][2] + rects[i - 1][3])) < 10))):
        label_ture.append(tag)  # 如果存在满足空间条件 以及左右标记都为tag 则设置标记为tag
    else:
        label_ture.append(tag_ver)
if label[i] == tag:
    label_ture.append(tag_ver)
label_ture = np.array(label_ture)

In [188]:
row_ture = []
for i in range(len(row)-1) :
    if row[i+1]-row[i] == 1:
        continue
    if rects[label==tag][row[i]][0] < 0.45*width and rects[label==tag][row[i+1]-1][0] > 0.55*width:
        row_ture.append([row[i],row[i+1]-1])

In [189]:
label_row = np.copy(label)
for i in row_ture:
    star = np.where(label==tag)[0][i[0]]    
    end = np.where(label==tag)[0][i[1]]
    for j in np.where(label_row==tag)[0]:
        if j  in range(star,end+1):
            label_row[j]=2
    for n in np.where(label_ture == tag)[0]:
        if n  in range(star,end+1):
            label_row[n]=2
label_row[label_row!=2]=tag_ver
label_row[label_row==2]=tag

In [190]:
new_rects=[]
copy = image.copy()
for i,rect in enumerate(rects[label_row==tag]):
    # 在事先做过膨胀操作很有可能 出现两个字母连在一起的可能
    # 在这里做一步判断排除这种可能
    if i!=0 and rects[label_row==tag][i][2] > 60 :
        x,y,w,h = rect
        new_w = w//2
        new_rects.append([x,y,w,h])
        cv2.rectangle(copy,(x,y),(x+w,y+h),(0,0,255),2)
        x,y,w,h = rect
        x+=w//2
        w=new_w
        cv2.rectangle(copy,(x,y),(x+w,y+h),(0,0,255),2)
        new_rects.append([x,y,w,h])
        continue
    x,y,w,h = rect
    new_rects.append([x,y,w,h])
    cv2.rectangle(copy,(x,y),(x+w,y+h),(0,0,255),2)

for i,rect in enumerate(new_rects):
    x,y,w,h = rect
    image_try = image[y:y+h,x:x+new_w]
    cv2.imshow(f"{i}",image_try)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [191]:
cv2.imshow("review",copy)
cv2.waitKey(0)
cv2.destroyAllWindows()